In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, Flatten
import os
import numpy as np

In [2]:
def read_data(filename):
    data=[]
    labels=[]
    with open(filename, 'r', encoding='latin-1') as f:
        temp=f.readlines()
        for row in temp:
            row_=row.split(':')
            row_[1]=row_[1].lower()
            data.append(row_[1].strip('\n').split())
            labels.append(row_[0])
            
    return data, labels

train_data, train_labels = read_data(os.getcwd()+'/train_1000.label')
test_data, test_labels = read_data(os.getcwd()+'/TREC_10.label')

In [3]:
label_ids={val:i for i,val in enumerate(set(train_labels))}

In [4]:
train_labels_enc=[label_ids[label] for label in train_labels]
test_labels_enc=[label_ids[label] for label in test_labels]
train_labels_=tf.one_hot(train_labels_enc, depth=6)
test_labels_=tf.one_hot(test_labels_enc, depth=6)

In [5]:
train_max=max([len(x) for x in train_data])
test_max=max([len(x) for x in test_data])

In [6]:
train_max

33

In [7]:
for qst in train_data:
    for _ in range(train_max-len(qst)):
        qst.append('<PAD>')
        
for qst in test_data:
    for _ in range(test_max-len(qst)):
        qst.append('<PAD>')

In [8]:
flattened_train = np.array(train_data).reshape(-1)
flattened_test = np.array(test_data).reshape(-1)
flattened=np.append(flattened_train, flattened_test)
vocabs=set(flattened)

In [9]:
vocab2int={c:i for i, c in enumerate(vocabs)}
int2vocab={i:c for i,c in zip(vocab2int.values(), vocab2int.keys())}
train_ids=[[vocab2int[word] for word in qst] for qst in train_data]
test_ids=[[vocab2int[word] for word in qst] for qst in test_data]
train_ids=np.array(train_ids)
train_ids=train_ids[:, :, np.newaxis]
test_ids=np.array(test_ids)
test_ids=test_ids[:, :, np.newaxis]

In [10]:
train_ids=tf.convert_to_tensor(train_ids, dtype=tf.float32)
test_ids=tf.convert_to_tensor(test_ids, dtype=tf.float32)

In [22]:
train_dataset=tf.data.Dataset.from_tensor_slices((train_ids, train_labels_)).shuffle(100).batch(32)
test_dataset=tf.data.Dataset.from_tensor_slices((test_ids, test_labels_)).shuffle(100).batch(32)

In [20]:
test_dataset

<BatchDataset shapes: ((None, 18, 1), (None, 6)), types: (tf.float32, tf.float32)>

In [12]:
class Model(tf.keras.Model):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.conv1=Conv1D(32, 3, padding='SAME', activation='relu')
        self.conv2=Conv1D(32, 5, padding='SAME', activation='relu')
        self.conv3=Conv1D(32, 7, padding='SAME', activation='relu')
        self.flatten=Flatten()
        self.dense=Dense(num_classes)
    
    def call(self,x):
        l1=self.conv1(x)
        l2=self.conv2(x)
        l3=self.conv3(x)
        l=tf.concat([tf.reduce_max(l1, axis=1), tf.reduce_max(l2, axis=1), tf.reduce_max(l3, axis=1)], axis=1)
        l=self.flatten(l)
        return self.dense(l)
    
model=Model(6) 

In [13]:
loss_object=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optm=tf.keras.optimizers.Adam()
train_loss=tf.keras.metrics.Mean()
train_acc=tf.keras.metrics.CategoricalAccuracy()
test_loss=tf.keras.metrics.Mean()
test_acc=tf.keras.metrics.CategoricalAccuracy()

@tf.function
def train_step(data, labels):
    with tf.GradientTape() as tape:
        predictions=model(data, training=True)
        loss=loss_object(labels, predictions)
    gradients=tape.gradient(loss, model.trainable_variables) 
    optm.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_acc(labels, predictions)
    
@tf.function
def test_step(data, labels):
    predictions=model(data)
    loss=loss_object(labels, predictions)
    
    test_loss(loss)
    test_acc(labels, predictions)
    


In [23]:
for epoch in range (50):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    
    for img, label in train_dataset:
        #print(img.shape)
        train_step(img, label)
        
    for img, label in test_dataset:
        test_step(img, label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_acc.result()*100,
                        test_loss.result(),
                        test_acc.result()*100))
    

Epoch 1, Loss: 13.460238456726074, Accuracy: 49.599998474121094, Test Loss: 18.637968063354492, Test Accuracy: 50.80000305175781
Epoch 2, Loss: 12.30281925201416, Accuracy: 54.29999923706055, Test Loss: 21.083158493041992, Test Accuracy: 50.0
Epoch 3, Loss: 13.582408905029297, Accuracy: 50.70000457763672, Test Loss: 19.457054138183594, Test Accuracy: 49.20000076293945
Epoch 4, Loss: 12.743268966674805, Accuracy: 52.79999923706055, Test Loss: 15.448917388916016, Test Accuracy: 36.0
Epoch 5, Loss: 12.718542098999023, Accuracy: 53.10000228881836, Test Loss: 22.753732681274414, Test Accuracy: 47.20000076293945
Epoch 6, Loss: 12.501111030578613, Accuracy: 53.29999923706055, Test Loss: 15.297054290771484, Test Accuracy: 53.20000076293945
Epoch 7, Loss: 11.791872024536133, Accuracy: 54.900001525878906, Test Loss: 15.758956909179688, Test Accuracy: 52.20000076293945
Epoch 8, Loss: 11.320253372192383, Accuracy: 53.79999923706055, Test Loss: 23.80356788635254, Test Accuracy: 51.0
Epoch 9, Loss: 